Comparing hydraulic conductivity values using K0 + L + Rosetta 1 parameters with values obtained using K0 + L=0.5 + Rosetta 3 parameters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MABarbadillo/
%cd 'GWP/GWP files'
import os

Mounted at /content/drive
/content/drive/MyDrive/MABarbadillo
/content/drive/MyDrive/MABarbadillo/GWP/GWP files


In [ ]:
##Install rosetta

!pip install rosetta-soil
from rosetta import rosetta, SoilData

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.5 MB/s eta 0:00:00


In [ ]:
import math
import numpy as np
import pandas as pd

# Read the .csv file 'MesoSoilv1_3.csv', skipping the first row and using the second row as header
df = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/MesoSoilv1_3.csv', header=0, skiprows=1)

# Extract the data from the specified columns
data = df.iloc[1:, [2, 3, 4, 5, 6, 7]].values.tolist()      #Columns 2,3,4,5,6, and 8 specify the columns for %sand, silt, clay, bulk density (g/cm3), and volumetric water contents (cm3/cm3) at -33 and -1500 kPa, respectively
K0 = df.iloc[1:, [13]].values.tolist()                      #K0 parameter from the csv file
L1 = df.iloc[1:, [14]].values.tolist()                       #L parameter from the csv file
L3 = 0.5   #default value

# Call the function to get hydraulic parameters
mean_1, stdev_1, codes_1 = rosetta(1, SoilData.from_array(data))    #for Rosetta Ver.1
mean_3, stdev_3, codes_3 = rosetta(3, SoilData.from_array(data))    #for Rosetta Ver.3

# Extract the hydraulic parameter values from the array using Rosetta Ver.1
theta_r1 = mean_1[0:, [0]].tolist()
theta_s1 = mean_1[0:, [1]].tolist()
alpha1 = mean_1[0:, [2]].tolist()
n1 = mean_1[0:, [3]].tolist()
m1 = 1 - (1 / n1)                   #########needs edit

# Extract the hydraulic parameter values from the array using Rosetta Ver.3
theta_r3 = mean_3[0:, [0]].tolist()
theta_s3 = mean_3[0:, [1]].tolist()
alpha3 = mean_3[0:, [2]].tolist()
n3 = mean_3[0:, [3]].tolist()
m3 = 1 - (1 / n3)                   #########needs edit



# To get the matric potential from Mesonet data
# Read .csv file '_____'.
df = pd.read_csv('Location/file.csv')
ref_Temperature60 = df.iloc[1:, [8]].values.tolist()   #Reference temperature at 60 cm is at column 8  /  Assuming that it is at the 2nd row
c = 0.717  #calibration constant [kPa]
a = 1.788  #calibration constant [1/C]
matric_potential = -c * np.exp(a * ref_Temperature60)  #[kPa]


# Convert matric potential to volumetric water content (vwc) for Rosetta Ver.1
vwc1 = theta_r1 + (theta_s1 - theta_r1) * (1 / (1 + (-alpha1 * matric_potential)**n1)**m1)    #cm3/cm3

# Convert matric potential to volumetric water content (vwc) for Rosetta Ver.3
vwc3 = theta_r3 + (theta_s3 - theta_r3) * (1 / (1 + (-alpha3 * matric_potential)**n3)**m3)    #cm3/cm3


# Convert volumetric water content to Effective saturation (Se) for Rosetta Ver.1
Se1 = (vwc1 - theta_r1)/(theta_s1 - theta_r1)    #cm3/cm3

# Convert volumetric water content to Effective saturation (Se) for Rosetta Ver.3
Se3 = (vwc3 - theta_r3)/(theta_s3 - theta_r3)    #cm3/cm3


# Calculate Hydraulic conductivity for Rosetta Ver.1
K_Se1 = (K0*Se1**L1) * (1 - (1-Se1**(n1/n1-1))**m1)**2    #cm/day

# Calculate Hydraulic conductivity for Rosetta Ver.3
# K0 values from 'MesoSoilv1_3.csv' were used
# L3 = 0.5
K_Se3 = (K0*Se3**L3) * (1 - (1-Se3**(n3/n3-1))**m3)**2    #cm/day


"""
##Codes from soilDrainage.m by Wyatt
##Still in matlab format

#Read in Mesonet *.csv file and create new excel file
#row1,col5 if downloading Temperature Reference AND Rainfall data
[mesoNum,mesoTxt,mesoRaw] = xlsread(mesonetfile);

#Read in Mesonet Temperature Reference values at 60cm and convert
#to hydraulic conductivity
#when reading in mesonet csv data, TR60 is at column 8
c=8;
TR60=mesoNum(:,c);

#find Mesonet error messages (all negative) and convert to NaN
i=TR60<0;
TR60(i)=nan;

#Convert Mesonet Temperature Reference Value to Matric Potential (MP)
MP=-.717*exp(1.788*TR60);%MP=kPa
c=1.788 degC^-1
a=.717kPa

#Convert Matric Potential to Volumetric Water Content (VWC)
VWC=ThetaR+((ThetaS-ThetaR)./((1+(-a*MP).^n).^(1-1/n))); %cm^3/cm^3

#Convert Volumetric Water Content to Effective Saturation (Se)
Se=(VWC-ThetaR)/(ThetaS-ThetaR); %cm^3/cm^3

#Convert Effective Saturation to Hydraulic Conductivity (HC)
HC=Ko*Se.^L.*(1-(1-Se.^(n/(n-1))).^(1-1/n)).^2; %cm/d
"""


"""
Print results
print("Mean:")
print(mean_1)
print(mean_3)
print("Standard Deviation:")
print(stdev_3)
print(stdev_1)
"""